## Author: Aditya Sundar
## Email: aditya.1094@gmail.com

# Modules

In [48]:
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
import time
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go

from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

# Web scraping to get data for electric vehicles (EV) sales in the USA
Data includes various types of electric vehicles, manufactured by several automobile companies

In [2]:
url = 'https://www.atlasevhub.com/materials/state-ev-registration-data/#data'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')

links=[link.get('href') for link in soup.find_all('a') ]
data=[i for i in links if len(i.split('/'))>1 and i.split('/')[1]=='public']
data=[i for i in data if i.split('.')[-1]=='csv']
state_name=[i.split('/')[3].split('_')[0].upper() for i in data]

dt=[]
for i in data:
    dt.append(pd.read_csv('https://www.atlasevhub.com/'+i))

# Data cleaning 
The data is binned annually from 2000-2022. The earliest recorded year among all datasets is 2003. The latest recorded year is 2022. For organisation into a dataframe, cumulative EV sales for the year without any recorded data are assigned the value of min(0, cumulative EV sales for the previous year).

In [9]:
def padding(df,n):
    df.loc[dtotal.shape[0]]=[2000,0,state_name[n]]
    for i in range(2000,2023):
        if i not in df['Date'].tolist():
            df.loc[df.shape[0]]=[i,df.loc[df['Date']==i-1]['Count'].values[0],state_name[n]]
    df=df.sort_values(by=['Date'])
    df=df.set_index(pd.Series(range(df.shape[0])))
    return df

dtotal=dt[0][['Registration Valid Date']]
dtotal['Registration Valid Date']=pd.to_datetime(dtotal['Registration Valid Date'])
dtotal.rename({'Registration Valid Date':'Date'},axis=1, inplace=True)
dtotal=dtotal.groupby(pd.to_datetime(dtotal['Date']).dt.year).count()
dtotal.rename({'Date':'Count'},axis=1, inplace=True)
dtotal['State']=state_name[0]
dtotal['Count']=dtotal['Count'].cumsum()
dtotal=dtotal.rename_axis('Date').reset_index()
dtotal=padding(dtotal,0)

for i in range(1,len(dt)):
    dtem=dt[i][['Registration Valid Date']]
    dtem['Registration Valid Date']=pd.to_datetime(dtem['Registration Valid Date'])
    dtem.rename({'Registration Valid Date':'Date'},axis=1, inplace=True)
    dtem=dtem.groupby(pd.to_datetime(dtem['Date']).dt.year).count()
    dtem.rename({'Date':'Count'},axis=1, inplace=True)
    dtem['State']=state_name[i]
    dtem['Count']=dtem['Count'].cumsum()
    dtem=dtem.rename_axis('Date').reset_index()
    dtem=padding(dtem,i)
    dtotal=pd.concat([dtotal,dtem],ignore_index=True)
    
dtotal['Date']=dtotal['Date'].astype(int)
dtotal.head()

# Export JSON file containing cumulative state-wise EV sales from 2000-2022
dtotal.to_json('dtotal.json',orient='records')

for i in range(len(state_name)):
    if 'Vehicle Name' in dt[i].columns:
        ds=dt[i][['Registration Valid Date','Vehicle Name']]
        ds['Vehicle Name']=[i.split()[0] for i in ds['Vehicle Name']]
        ds.rename(columns = {'Registration Valid Date':'Date','Vehicle Name':'Make'}, inplace = True)
    elif 'Make' in dt[i].columns:
        ds=dt[i][['Registration Valid Date','Make']]
        ds['Make']=[i.split()[0] for i in ds['Make']]
        ds.rename(columns = {'Registration Valid Date':'Date','Make':'Make'}, inplace = True)
    ds.to_json(state_name[i]+'_data.json',orient='records')

# Visualisation of EV sales: state-wise and year-wise

In [465]:
dtotal=pd.read_json('dtotal.json', orient='records')
fig = px.choropleth(dtotal,
                    locations='State', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='Count',
                    color_continuous_scale="Viridis_r",
                    animation_frame = 'Date' 
                    )
fig.update_layout(
            title_text="Cumulative Electric Vehicle sales in the United States",
            title_xanchor="center",
            title_font=dict(size=20),
            title_x=0.5,
            title_y=0.95,
            geo=dict(scope='usa'),
            width=700,
            height=500,
            margin=dict(t=100, b=0,r=100,  l=0)
            )

fig.update_layout(sliders=[{"currentvalue": {"prefix": "Year="}}])
fig.show()

# New EV registered per year

In [453]:
n=pd.read_json(state_name[0]+'_data.json', orient='records')
n['Date']=n['Date'].dt.year
n['State']=[state_name[0]]*n['Date'].shape[0]
for i in state_name[1:]:
    t=pd.read_json(i+'_data.json', orient='records')
    t['Date']=t['Date'].dt.year
    t['State']=[i]*t['Date'].shape[0]
    n=pd.concat((n,t))

In [469]:
dropdown_s=widgets.Dropdown(
    options=state_name,
    value='MI',
    description='State:',
    disabled=False,
)
dropdown_y=widgets.Dropdown(
    options=list(range(2000,2023,1)),
    value=2020,
    description='Year:',
    disabled=False,
)

In [471]:
def plot_func(s,y):
    nsy=n[(n['State']==s) & (n['Date']==y)]
    fig = go.Figure()
    fig = px.histogram(nsy,x='Make',title='EV sales in '+s+' in '+str(y)).update_xaxes(categoryorder='total descending')
    fig.update_layout(width=700,height=500,margin=dict(t=30, b=0,r=0,  l=0))
    fig.show()

interact(plot_func, s = dropdown_s, y=dropdown_y);

interactive(children=(Dropdown(description='State:', index=5, options=('CA', 'CO', 'CT', 'FL', 'MT', 'MI', 'MN…

# Web scraping to get data for alternative fuel stations in the USA

In [ ]:
driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()))

In [ ]:
driver.get('https://afdc.energy.gov/data_download')
Select(driver.find_element('xpath','//*[@id="download_data_api"]')).select_by_visible_text('Alternative fuel stations')
Select(driver.find_element('xpath','//*[@id="download_data_format"]')).select_by_visible_text('CSV (opens in Excel)')
Select(driver.find_element('xpath','//*[@id="download_data_api_options_alt_fuel_stations_fuel_type"]')).select_by_visible_text('All')
Select(driver.find_element('xpath','//*[@id="download_data_api_options_alt_fuel_stations_access"]')).select_by_visible_text('All')
Select(driver.find_element('xpath','//*[@id="download_data_api_options_alt_fuel_stations_status"]')).select_by_visible_text('All')
driver.find_element('xpath','//*[@id="download_api_user_first_name"]').send_keys('Aditya')
driver.find_element('xpath','//*[@id="download_api_user_last_name"]').send_keys('Sundar')
driver.find_element('xpath','//*[@id="download_api_user_email"]').send_keys('aditya.1094@gmail.com')
driver.find_element('xpath','//*[@id="download_api_user_terms_and_conditions"]').click()
time.sleep(3)
driver.find_element('xpath','//*[@id="main_content"]/form/input[3]').send_keys(Keys.ENTER)

In [526]:
fuels=pd.read_csv('alt_fuel_stations (Aug 10 2022).csv',dtype='object')
ftype=fuels['Fuel Type Code'].unique()
marker_c=['black','blue','red','green','brown','purple','orange']

In [567]:
dropdown_ftype=widgets.Dropdown(
    options=ftype,
    value=ftype[0],
    description='Fuel Type:',
    disabled=False,
)

def plot_func(f):
    ff=fuels[fuels['Fuel Type Code']==f]
    fig = go.Figure(data=go.Scattergeo(
        locationmode = 'USA-states',
        lon = ff['Longitude'],
        lat = ff['Latitude'],
        text = ff['City']+', '+ff['State'],
        mode = 'markers',
        hoverinfo='text',
        marker = dict(
            size = 4,
            symbol = 'square',
            color=marker_c[np.where(ftype==f)[0][0]])))

    fig.update_layout(
            title = 'Alternative fuel stations in the USA',
            geo = dict(
                scope='usa',
                projection_type='albers usa',
                showland = True,
                landcolor = "rgb(245, 245, 245)",
                subunitcolor = "rgb(217, 217, 217)",
                countrycolor = "rgb(217, 217, 217)",
                countrywidth = 0.5,
                subunitwidth = 0.5,
                
            ),width=700,height=500,margin=dict(t=40, b=0,r=0,  l=0)
        )
    fig.show()
    
interact(plot_func, f = dropdown_ftype);

interactive(children=(Dropdown(description='Fuel Type:', options=('CNG', 'E85', 'ELEC', 'LNG', 'LPG', 'BD', 'H…